In [1]:
import json

joystick_data = json.load(open("selected_trials.json"))



In [2]:
joystick_data[0]

{'ID': '2',
 'SI': {'Swing': {'Swing': None,
   'Bucket': 0.007272727272727273,
   'Arm': 0.046255506607929514,
   'Boom': 0.03308270676691729},
  'Bucket': {'Swing': 0.007272727272727273,
   'Bucket': None,
   'Arm': 0.13506493506493505,
   'Boom': 0.2700587084148728},
  'Arm': {'Swing': 0.046255506607929514,
   'Bucket': 0.13506493506493505,
   'Arm': None,
   'Boom': 0.13095238095238096},
  'Boom': {'Swing': 0.03308270676691729,
   'Bucket': 0.2700587084148728,
   'Arm': 0.13095238095238096,
   'Boom': None}},
 'BCS': 0.07659318751424014,
 'control_usage': {'2_controls': 24.452133794694348,
  '3_controls': 5.074971164936563,
  '4_controls': 0.11534025374855825},
 'URL': 'https://youtu.be/9at52tHTOLo'}

In [3]:
import json
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

# Load the data
joystick_data = json.load(open("selected_trials.json"))

# Function to create SI Heatmap for a specific trial using Plotly
def create_si_heatmap(trial_data, output_path="SI_Heatmap.png"):
    """
    Create a Simultaneity Index (SI) heatmap from trial data using Plotly
    """
    si_data = trial_data['SI']
    controls = ['Swing', 'Bucket', 'Arm', 'Boom']
    
    # Create matrix and corresponding annotation labels
    matrix = np.zeros((4, 4))
    for i, control1 in enumerate(controls):
        for j, control2 in enumerate(controls):
            if i == j:
                matrix[i, j] = np.nan  # Diagonal is NaN (not shown)
            else:
                matrix[i, j] = si_data[control1][control2]

    fig = px.imshow(
        matrix,
        x=controls,
        y=controls,
        color_continuous_scale='Blues',
        text_auto='.3f',
        aspect='auto',
        labels=dict(color="Simultaneity Index (SI)"),
        range_color=[0, 1]
    )
    # Mask diagonal (hide) by making it fully transparent
    for i in range(4):
        fig.add_shape(type="rect",
                      x0=i-0.5, x1=i+0.5,
                      y0=i-0.5, y1=i+0.5,
                      fillcolor="white",
                      line=dict(color="rgba(255,255,255,0)"),
                      layer='above')
    fig.update_layout(
        title=dict(text='Simultaneity Index (SI) Heatmap', font=dict(size=18, family='Arial', color='#333'), x=0.5),
        width=650,
        height=550,
        font=dict(size=14),
        xaxis=dict(title='', side='top'),
        yaxis=dict(title='')
    )
    # Make text on diagonal cells blank
    z_text = np.empty(matrix.shape, dtype='<U8')
    for i in range(4):
        for j in range(4):
            if i == j:
                z_text[i, j] = ""
            else:
                z_text[i, j] = f"{matrix[i, j]:.3f}"
    # Update text annotations
    fig.update_traces(text=z_text, textfont={"size":16, "family":"Arial"})
    # Save as static image
    fig.write_image(output_path, scale=2)
    print(f"SI Heatmap saved to {output_path}")

# Function to create Control Usage bar chart for a specific trial using Plotly
def create_control_usage_chart(trial_data, output_path="control_usage.png"):
    """
    Create a Simultaneous Control Usage bar chart from trial data using Plotly
    """
    control_usage = trial_data['control_usage']
    
    # Prepare data
    categories = ['≥2 controls', '≥3 controls', '4 controls']
    values = [
        control_usage['2_controls'],
        control_usage['3_controls'],
        control_usage['4_controls']
    ]
    
    fig = go.Figure(
        data=[
            go.Bar(
                x=categories,
                y=values,
                marker=dict(color='#4361ee'),
                text=[f"{v:.1f}%" for v in values],
                textposition='outside',
                textfont=dict(size=16, family='Arial', color='#222'),
            )
        ]
    )
    fig.update_layout(
        yaxis_title='Percentage of Time (%)',
        xaxis_title=None,
        title=dict(text="Simultaneous Control Usage", x=0.5, font=dict(size=18)),
        yaxis=dict(range=[0, 100], gridcolor='rgba(0,0,0,0.08)', gridwidth=1, zeroline=False, showticklabels=True),
        xaxis=dict(showticklabels=True),
        width=500,
        height=400,
        font=dict(size=14),
        margin=dict(t=80, l=60, r=60, b=40),
        plot_bgcolor="#fff"
    )
    fig.update_traces(marker_line_color='rgba(70,70,70,0.18)', marker_line_width=1)
    fig.write_image(output_path, scale=2)
    print(f"Control Usage chart saved to {output_path}")



In [4]:
# Generate charts for all trials
import os

# Create folders for SI heatmaps and Control Usage charts if they don't exist
si_folder = "SI_Heatmaps"
control_usage_folder = "Control_Usage"

os.makedirs(si_folder, exist_ok=True)
os.makedirs(control_usage_folder, exist_ok=True)

for trial in joystick_data:
    trial_id = trial['ID']
    print(f"\nGenerating charts for trial {trial_id}...")
    si_output_path = os.path.join(si_folder, f"SI_Heatmap_{trial_id}.png")
    control_usage_output_path = os.path.join(control_usage_folder, f"control_usage_{trial_id}.png")
    create_si_heatmap(trial, si_output_path)
    create_control_usage_chart(trial, control_usage_output_path)


Generating charts for trial 2...
SI Heatmap saved to SI_Heatmaps/SI_Heatmap_2.png
Control Usage chart saved to Control_Usage/control_usage_2.png

Generating charts for trial 21...
SI Heatmap saved to SI_Heatmaps/SI_Heatmap_21.png
Control Usage chart saved to Control_Usage/control_usage_21.png

Generating charts for trial 31...
SI Heatmap saved to SI_Heatmaps/SI_Heatmap_31.png
Control Usage chart saved to Control_Usage/control_usage_31.png

Generating charts for trial 52...
SI Heatmap saved to SI_Heatmaps/SI_Heatmap_52.png
Control Usage chart saved to Control_Usage/control_usage_52.png

Generating charts for trial 67...
SI Heatmap saved to SI_Heatmaps/SI_Heatmap_67.png
Control Usage chart saved to Control_Usage/control_usage_67.png

Generating charts for trial A9...
SI Heatmap saved to SI_Heatmaps/SI_Heatmap_A9.png
Control Usage chart saved to Control_Usage/control_usage_A9.png

Generating charts for trial A10...
SI Heatmap saved to SI_Heatmaps/SI_Heatmap_A10.png
Control Usage chart sa